<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2020, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-02-04 15:03:02  
@LastEditors: Liusen  
@LastEditTime: 2020-02-04 15:03:02   

以下主要用说明： 
  
  建议我们用模拟模式，进入模拟模式按下手柄ANALOG按键，对应下面的红灯亮表示进入成功。


上图滑竿0-9表示的是模拟值：取值方法：controller.axes[0].value = -1.0 ~ 1.0  
上图方块0-11表示的按键的Bool值：取值方法： controller.buttons[0].value = True / False  



In [24]:
from Raspblock import Raspblock
import ipywidgets.widgets as widgets
import time

# 线程功能操作库
import threading
import inspect
import ctypes
import matplotlib.pyplot as plt


robot = Raspblock()
# joystick is a widgets of jpuyter
controller = widgets.Controller(index=0)  #用上面手柄检测方法，网页测试出来的索引号替代
display(controller)                       #显示手柄控件



serial Open!
serial Close!


Controller()

In [25]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [26]:
'''以下是舵机控制的部分    '''
global leftrightpulse
leftrightpulse = 1500
global updownpulse
updownpulse = 1500

def camUpFunction():
    global updownpulse
    updownpulse+=10
    if updownpulse>2500:
        updownpulse=2500
    robot.Servo_control(leftrightpulse, updownpulse)

def camDownFunction():
    global updownpulse
    updownpulse-=10
    if updownpulse<500:
        updownpulse=500
    robot.Servo_control(leftrightpulse, updownpulse)

def camLeftFunction():
    global leftrightpulse
    leftrightpulse+=10
    if leftrightpulse>2500:
        leftrightpulse=2500
    robot.Servo_control(leftrightpulse, updownpulse)

def camRightFunction():
    global leftrightpulse
    leftrightpulse-=10
    if leftrightpulse<500:
        leftrightpulse=500
    robot.Servo_control(leftrightpulse, updownpulse)

def camservoInitFunction():
    global leftrightpulse, updownpulse
    leftrightpulse = 1500
    updownpulse = 1500
    robot.Servo_control(leftrightpulse, updownpulse)

In [27]:
def Remote_thread():
    Speed_axis_X = 0
    Speed_axis_Y = 0
    Speed_axis_Z = 0

    count1 = count2 = count3 = count4 =  count5 = count6 = count7 = 0
    while 1:
        #手柄操作代码---(亚博配手柄) for car, axes[0]: -left +right, axes[1]: +up -down
        if(abs(controller.axes[0].value) >= 0.01 or abs(controller.axes[1].value) >= 0.01):
            count4 += 1
            if(count4 >= 1):
                Speed_axis_X = int(controller.axes[0].value * 4)
                Speed_axis_Y = -int(controller.axes[1].value * 4)
                robot.Speed_axis_Yawhold_control(Speed_axis_X,Speed_axis_Y)
                count4 = 0
        time.sleep(0.01)
        
        #蜂鸣器控制：数字1按键
        if controller.buttons[0].value == True:
            robot.Buzzer_control(1) 
        else:
            robot.Buzzer_control(0)
            
        #摄像头上下舵机： 右摇杆上下
        # axes[2] left right, axes[3] up down
        # yahboom's original source code misplace the [2] and [3]
        # mark by wang yong 2023.11.28
        if controller.axes[3].value == -1:
            count1  += 1
            if count1 >= 1:
                camDownFunction()
                count1 = 0
        elif controller.axes[3].value == 1:
            count1  += 1
            if count1 >= 1:
                camUpFunction()
                count1 = 0
        else:
            count1 = 0
        
        #摄像头左右舵机    右摇杆左右
        # if controller.axes[5].value == 1:  why [5]? the largest axes is 3
        # sometime it shows 8 axes, the up down is on axes[5]
        if controller.axes[2].value == 1:    
            count2  += 1
            if count2 >= 1:
                camRightFunction()
                count2 = 0
        # elif controller.axes[5].value == -1:
        elif controller.axes[2].value == -1:
            count2  += 1
            if count2 >= 1:
                camLeftFunction()
                count2 = 0
        else:
            count2 = 0
        
        #摄像头上下左右舵机 复位： 右摇杆按下
        if controller.buttons[11].value == True:
            count3 += 1
            if count3 >= 1:
                camservoInitFunction()
                count3 = 0
        else:
            count3 = 0
            
        #自旋：数字4（左旋） 数字2（右旋）
        if controller.buttons[3].value == True:
            count6 += 1
            if count6 >= 3:
                robot.Speed_axis_control(0,0,-15)
                count6 = 0
        elif controller.buttons[1].value == True:
            count6 += 1
            if count6 >= 3:
                robot.Speed_axis_control(0,0,15)
                count6 = 0
        else:
            count6 = 0
            
        time.sleep(0.01)
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [28]:
thread = threading.Thread(target=Remote_thread)
thread.setDaemon(True)
thread.start()

In [17]:
print("start joystick")

start joystick


In [18]:
stop_thread(thread)
del robot

serial Close!
